# Requête HTTP 

Un requête HTTP est une requête basée sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisés dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas, nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme son nom l'indique de récupérer des informations en fonction de certains paramètres. 
- Le POST nécessite un envoi de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [14]:
import requests

In [15]:
url = "https://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [18]:
type(response.content)

bytes

In [19]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [7]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [20]:
response.headers

{'Date': 'Tue, 19 Nov 2024 12:10:37 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '16453', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '108036618 109347314', 'Age': '7', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramètres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent (identifiant de l'initiateur de la requête) et un timeout de 10 secondes :

In [21]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2024 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [36]:
# EXERCICE 1

class HTTPClient:
    
    def __init__(self, max_retries=3):
        # Déf user-agent
        self.user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        # Nombre de retries autorisés avant d'abandonner la requête (par défaut, 3 tentatives).
        self.max_retries = max_retries
        
        
    def make_request(self, url, timeout=10, retry_count=0):
        headers = {
            'User-Agent': self.user_agent
        }
        
        try:
            # Tentative d'envoi de la requête GET avec l'URL et les headers
            response = requests.get(url, headers=headers, timeout=timeout)
            # Vérification du statut de la réponse. Si erreur, une exception est levée.
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            # Si une erreur se produit (connexion, timeout, etc.), elle est capturée ici
            print(f"Erreur lors de la requête : {e}")
            
            # Si on n'a pas atteint le nombre maximal de retries, on tente à nouveau
            if retry_count < self.max_retries:
                print(f"Retry {retry_count + 1}/{self.max_retries}")
                return self.make_request(url, timeout, retry_count + 1)
            else:
                # Si le nombre de retries a été atteint, on abandonne
                print("Nombre maximal de retries atteint.")
                return None
            
    # Méthode de test pour effectuer une requête HTTP et afficher le résultat
    def test_request(self, url, timeout=10):
        # Affichage de l'URL avant de faire la requête
        print("Test de requête HTTP avec URL :", url)
        
        # Appel de la méthode make_request pour obtenir la réponse
        response = self.make_request(url, timeout)
        
        # Si la requête a réussi, on affiche le code de statut de la réponse
        if response:
            print("Test réussi, statut de la réponse :", response.status_code)
        else:
            # Si la requête a échoué après plusieurs tentatives, on affiche un message d'erreur
            print("Test échoué après plusieurs retries.")


In [45]:
client = HTTPClient(max_retries=3)

# Test normal avec timeout par défaut
client.test_request("https://www.esiee.fr/")
print('\n')

# Test avec un timeout court pour forcer un retry
client.test_request("https://www.esiee.fr/", timeout=0.001)

Test de requête HTTP avec URL : https://www.esiee.fr/
Test réussi, statut de la réponse : 200


Test de requête HTTP avec URL : https://www.esiee.fr/
Erreur lors de la requête : HTTPSConnectionPool(host='www.esiee.fr', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10be2d580>, 'Connection to www.esiee.fr timed out. (connect timeout=0.001)'))
Retry 1/3
Erreur lors de la requête : HTTPSConnectionPool(host='www.esiee.fr', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10c105c10>, 'Connection to www.esiee.fr timed out. (connect timeout=0.001)'))
Retry 2/3
Erreur lors de la requête : HTTPSConnectionPool(host='www.esiee.fr', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10c106510>, 'Connection to www.esiee.fr timed out. (connect timeout=0.001)'))
Retry 3/3
Erreur lor

In [26]:
# EXERCICE 2

def remove_spaces(string):
    
    return " ".join(string.split())

In [27]:
def remove_special_carac_html_string(html):
    
    # initializing bad_chars_list
    bad_chars = [';', ':', '!', "*", " "]
        
    # remove bad_chars
    for i in bad_chars:
        new_string = html.replace(i, '')
        
    return new_string

In [29]:
def extract_domain(url):
    
    from urllib.parse import urlparse
    
    parsed_url = urlparse(url)
    return parsed_url.netloc


# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, il est normalement déjà installé, le cas échéant executez les lignes suivantes : 

In [30]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [31]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Pour trouver tous les liens d'une page, on récupère la balise `a` qui permet de gérer les liens en HTML :

In [32]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/actualites/journees-portes-ouvertes-2024-2025" target="_blank" title="Ouvre une nouvelle fenêtre">Bien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 7 décembre de 13h à 18h pour notre journée portes ouvertes.</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut aussi préciser la classe HTML qu'on veut récupérer :

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [33]:
soup.find_all(class_="toggler")[0:5]

[<button aria-controls="searchbox-header-form" aria-expanded="false" class="toggler">
 <i class="fa-solid fa-magnifying-glass"></i>
 <i class="fa-solid fa-xmark"></i>
 <span class="sr-only">
 <span class="display">Afficher</span><span class="hide">Masquer</span> la recherche
 		</span>
 </button>,
 <button aria-controls="submenu-40" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>L'école</button>,
 <button aria-controls="submenu-563" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Gouvernance et conseils</button>,
 <button aria-controls="submenu-65" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Départements d'enseignements et de recherche</button>,
 <button aria-controls="su

Pour récupérer le text sans les balises HTML :

In [34]:
soup.text[0:1000]

"\n\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer ! Rendez-vous à ESIEE Paris le 7 décembre de 13h à 18h pour notre journée portes ouvertes.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnem

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

In [70]:
# EXERCICE 3 

from urllib.parse import urlparse, urljoin
import random

class HTTPClient:
    
    def __init__(self, max_retries=3):
    
        # Def user agent
        self.user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        # Nb de retry
        self.max_retries = max_retries
    
    # effectuer une requête
    def make_request(self, url, timeout=10, retry_count=0):
        headers = {
            'User-Agent': self.user_agent
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"Erreur lors de la requête : {e}")
            
            if retry_count < self.max_retries:
                print(f"Retry {retry_count + 1}/{self.max_retries}")
                return self.make_request(url, timeout, retry_count + 1)
            else:
                print("Nombre maximal de retries atteint.")
                return None
    
    def test_request(self, url, timeout=10):
        print("Test de requête HTTP avec URL :", url)
        
        response = self.make_request(url, timeout)
        
        if response:
            print("Test réussi, statut de la réponse :", response.status_code)
        else:
            print("Test échoué après plusieurs retries.")
            
            
            
    # rotation aléatoire sur l'user agent  
    def rotate_user_agent(self):
        """Choisit un User Agent aléatoire dans la liste."""
        return random.choice(self.user_agents)
    
    # récupérer l'objet soup d'un url
    def get_soup(self, url, timeout=10):
        """Récupère un objet BeautifulSoup à partir d'une URL."""
        response = self.make_request(url, timeout)
        if response:
            return BeautifulSoup(response.text, 'html.parser')
        else:
            return None

    # parser la page HTML
    def parse_page(self, url, timeout=10):
        """Récupère les éléments principaux d'une page HTML."""
        soup = self.get_soup(url, timeout)
        if not soup:
            return None

        # Extraction des données demandées
        data = {
            "title": soup.title.string if soup.title else "Titre non trouvé",
            "h1_tags": [h1.get_text(strip=True) for h1 in soup.find_all('h1')],
            "image_links": [img['src'] for img in soup.find_all('img', src=True)],
            "external_links": [],
            "main_text": soup.get_text(strip=True),
        }

        # Extraction des liens externes
        all_links = soup.find_all('a', href=True)
        domain = urlparse(url).netloc
        for link in all_links:
            href = link['href']
            if urlparse(href).netloc and urlparse(href).netloc != domain:
                data["external_links"].append(urljoin(url, href))

        return data 

In [71]:
client = HTTPClient()

url = "https://www.esiee.fr/"

parsed_data = client.parse_page(url)
if parsed_data:
    print("Titre de la page :", parsed_data["title"])
    print("\nH1 tags :", parsed_data["h1_tags"])
    print("\nLiens vers des images :", parsed_data["image_links"])
    print("\nLiens externes :", parsed_data["external_links"])
    print("\nTexte principal (extrait) :", parsed_data["main_text"][:200], "...")
else:
    print("Impossible de parser la page.")

Titre de la page : ESIEE Paris, l'école de l'innovation technologique | ESIEE Paris

H1 tags : ['']

Liens vers des images : ['/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/user_upload/Fichiers/image-home/ESIEE-Home-Main-Picture.webp', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/9/5/csm_actu-jpo-1344x840_6889669208.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/0/b/csm_photos-salons-1344x840_472aff3f3e.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/0/3/csm_sandrine-wamy-trophees-des-femmes-de-l_industrie-2024-1344x840_7f464ae034.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw==', '/fileadmin/_processed_/1/6/csm_illustrat

# Exploitation des appels d'API



Losque le front du site récupère des données sur une API gérée par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL de la requête et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certains paramètres. L'API vérifie que dans les headers de la requête HTTP ces paramètres sont présents :
* un token généré à la volée avec des protocoles OAuth2 (ou moins développés).
* un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Google. 

Tips : 

- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
- Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises

In [131]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

class HTTPClientGoogle:
    
    # Constructeur de classe 
    def __init__(self, max_retries=3):
        self.user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        self.max_retries = max_retries
        
    # Méthode pour effectuer la requête HTTP
    def make_request(self, url, timeout=10, retry_count=0):
        headers = {
            'User-Agent': self.user_agent
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"Erreur lors de la requête : {e}")
            if retry_count < self.max_retries:
                print(f"Retry {retry_count + 1}/{self.max_retries}")
                return self.make_request(url, timeout, retry_count + 1)
            else:
                print("Nombre maximal de retries atteint.")
                return None

    # Récupérer et analyser le contenu HTML de la page
    def get_soup(self, url, timeout=10):
        response = self.make_request(url, timeout)
        if response:
            return BeautifulSoup(response.text, 'html.parser')
        return None
    
    # Méthode pour effectuer une recherche sur Google
    def google_search(self, query, num_results=10):
        base_url = "https://www.google.com/search"
        results = []

        params = {
            'q': query,
            'num': num_results
        }
        
        search_url = f"{base_url}?q={query}&num={num_results}"
        soup = self.get_soup(search_url, timeout=10)
        if not soup:
            print("Erreur lors du scraping de Google")
            return None

        for result in soup.find_all('div', class_='tF2Cxc'):
            title = result.find('h3').get_text() if result.find('h3') else "Titre non trouvé"
            link = result.find('a')['href'] if result.find('a') else "Lien non trouvé"
            link = urljoin(base_url, link)  
            snippet = result.find('span', class_='aCOpRe').get_text() if result.find('span', class_='aCOpRe') else "Description non trouvée"
            results.append({'title': title, 'link': link, 'snippet': snippet})
            
        return results


client = HTTPClientGoogle()

query = "ESIEE"
results = client.google_search(query, num_results=5)

if results:
    for result in results:
        print(result)
else:
    print("Aucun résultat trouvé.")


Aucun résultat trouvé.


# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles

In [139]:
import requests
import feedparser

class RSSClient:
    def __init__(self, max_retries=3):
        self.user_agent = (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        )
        self.max_retries = max_retries

    def make_request(self, url, timeout=5, retry_count=0):
        headers = {"User-Agent": self.user_agent}
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"Erreur lors de la requête : {e}")
            if retry_count < self.max_retries:
                print(f"Retry {retry_count + 1}/{self.max_retries}")
                return self.make_request(url, timeout, retry_count + 1)
            else:
                print("Nombre maximal de retries atteint.")
                return None

    def get_rss_data(self, url, timeout=10):
        """Récupère et analyse le flux RSS avec feedparser."""
        response = self.make_request(url, timeout)
        if response:
            # Utiliser feedparser pour analyser le contenu RSS
            feed = feedparser.parse(response.content)

            # Vérifier si le flux est valide
            if feed.bozo == 1:
                print("Erreur lors du parsing du flux RSS.")
                return None

            return self.parse_rss(feed)
        else:
            return None

    def parse_rss(self, feed):
        """Extrait les données du flux RSS analysé."""
        items = []
        for entry in feed.entries:
            title = entry.get("title", "Titre non trouvé")
            link = entry.get("link", "Lien non trouvé")
            description = entry.get("description", "Description non trouvée")
            category = entry.get("category", "Catégorie non trouvée")
            domain = link.split("/")[2] if link else "Domaine non trouvé"

            item_data = {
                "title": title,
                "link": link,
                "description": description,
                "category": category,
                "domain": domain,
            }
            items.append(item_data)

        return items


# URL correcte pour un flux RSS du Monde
url = "https://www.lemonde.fr/rss/en_continu.xml"
rss_client = RSSClient()
articles = rss_client.get_rss_data(url)

if articles:
    for article in articles:
        print(article)
else:
    print("Aucun article trouvé.")


{'title': 'Après Michelin, ArcelorMittal envisage la fermeture de deux sites en France', 'link': 'https://www.lemonde.fr/economie/article/2024/11/19/apres-michelin-arcelormittal-envisage-la-fermeture-de-deux-sites-en-france_6403694_3234.html', 'description': 'L’annonce pourrait provoquer la perte d’environ 130\xa0emplois, une centaine à Reims et une trentaine à Denain, ont rapporté deux syndicats. Alors que le site de Denain se met en grève immédiatement, des actions sont prévues\xa0sur tous les sites\xa0ArcelorMittal lundi et mardi.', 'category': 'Catégorie non trouvée', 'domain': 'www.lemonde.fr'}
{'title': 'Au fait, qu’est-ce qu’une espèce\xa0? Comment l’étude de l’ADN bouleverse la classification du vivant', 'link': 'https://www.lemonde.fr/sciences/article/2024/11/19/au-fait-qu-est-ce-qu-une-espece_6403384_1650684.html', 'description': 'Tout le monde croit pouvoir répondre à cette question. Pourtant, les définitions varient selon les spécialités scientifiques. Les chercheurs navigu

In [142]:
import requests
from bs4 import BeautifulSoup

class HTMLClient:
    def __init__(self, max_retries=3):
        self.user_agent = (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        )
        self.max_retries = max_retries

    def make_request(self, url, timeout=5, retry_count=0):
        headers = {"User-Agent": self.user_agent}
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.RequestException as e:
            print(f"Erreur lors de la requête : {e}")
            if retry_count < self.max_retries:
                print(f"Retry {retry_count + 1}/{self.max_retries}")
                return self.make_request(url, timeout, retry_count + 1)
            else:
                print("Nombre maximal de retries atteint.")
                return None

    def get_html_data(self, url, timeout=10):
        """Récupère et analyse le contenu HTML."""
        response = self.make_request(url, timeout)
        if response:
            # Utiliser BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(response.content, "html.parser")
            return self.parse_html(soup)
        else:
            return None

    def parse_html(self, soup):
        """Extrait les données du contenu HTML."""
        data = []

        for article in soup.find_all("article"):  
            title = article.find("h2").text if article.find("h2") else "Titre non trouvé"
            link = article.find("a")["href"] if article.find("a") else "Lien non trouvé"
            description = (
                article.find("p").text if article.find("p") else "Description non trouvée"
            )

            item_data = {
                "title": title,
                "link": link,
                "description": description,
            }
            data.append(item_data)

        return data


url = "https://www.lemonde.fr/"  
html_client = HTMLClient()
articles = html_client.get_html_data(url)

if articles:
    for article in articles:
        print(article)
else:
    print("Aucun article trouvé.")


Aucun article trouvé.
